<div class="alert alert-block alert-info">
<center> <h1> PORTUGUESE MEALS IMAGE RECOGNITION </h1> </center> <br>
<center> DEEP LEARNING NEURAL NETWORKS | FALL SEMESTER 2022/ 2023 </center>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**GROUP 10:** <br>

- João Magalhães      `20211044` <br>
- Maria Trindade      `20211049` <br>
- Nuno Bolas          `20211052` <br>
- Jorge Gaspar        `20211057` <br> 
- Mariana Teixeira    `20211058` <br>

<a id='toc'></a>


**Table of Contents** <br>


[BACK TO TOC](#toc)
    
<a id='import'></a>

# <font color = 'darkblue'> 1. Import</font>

<a class="anchor" id="importlibraries">

## 1.1. Import Libraries

</a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, layers, initializers, regularizers, optimizers, metrics 
from keras import callbacks

import os
import os.path
import pathlib
from pathlib import Path
import time
from datetime import datetime
import io
import itertools
from packaging import version
import random 
import zipfile
import shutil
from shutil import copyfile

import numpy as np
import pandas as pd

import PIL
import PIL.Image

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.colors import ListedColormap

import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
# Set the style for the Seaborn's plots
sns.set_style('whitegrid',{
    'xtick.bottom': False,
    'xtick.color': '.1',
    'xtick.direction': 'out',
    'xtick.top': False,
    'xtick.major.size': 1,
    'xtick.minor.size': 0.5,
    'ytick.left': True,
    'ytick.color': '.1',
    'ytick.direction': 'out',
    'ytick.right': False,
    'ytick.major.size': 1,
    'ytick.minor.size': 0.5,    
    'ytick.color': '.1',
    'grid.linestyle': '--',
    'axes.edgecolor': '.1',
    'grid.color': '0.8'
 })

palette = sns.color_palette("Set2") 
sns.set_palette(palette)

<a class="anchor" id="setuptensorboard">

## 1.2. Setup Tensorboard

</a>

Code inspired by: https://neptune.ai/blog/tensorboard-tutorial

In [ ]:
%load_ext tensorboard

In [ ]:
#Folder to save the logs
log_folder = 'logs'

In [ ]:
%reload_ext tensorboard

In [ ]:
#Clear previous logs

#collab
!rm -rf /logs/ 


#jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
#try:
#    shutil.rmtree('logs')
#except:
#    pass

In [ ]:
import datetime
log_folder = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

[BACK TO TOC](#toc)
    
<a id='dataunderstanding'></a>

# <font color = 'darkblue'> 2. Data Understanding and Preprocessing </font>

<a class="anchor" id="loaddata">

## 2.1. Data Loading

</a>

In [ ]:
#Defining files path

path = '../Dataset/dataset_dishes'

try: 
    os.chdir(path)
except: 
    print('Curret path is: ' + os.getcwd())
    pass

Download the dataset of the dishes photos following the tensorflow tutorial for loading images https://www.tensorflow.org/tutorials/load_data/images and the notebook shown in class (t_w3_2022_drive)

The dataset contains 6 subdirectories corresponding to each class.

In [ ]:
data_dir = os.getcwd()
data_dir_train = pathlib.Path(os.path.join(data_dir, "training"))
data_dir_val = pathlib.Path(os.path.join(data_dir, "validation"))
data_dir_test= pathlib.Path(os.path.join(data_dir, "test"))

In [ ]:
data_dir_train

In [ ]:
image_count = len(list(data_dir_train.glob('*/*.jpg')))
print(image_count)

Create a dataframe and check the classes' balance

In [ ]:
filepaths = list(data_dir_train.glob(r'**/*.jpg')) #iterate over folders and sub-folders
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths)) #Create labels with the number of classes

filepaths = pd.Series(filepaths, name='Filepath').astype(str) #create filepaths column
labels = pd.Series(labels, name='Label') #create classes names column

df = pd.concat([filepaths, labels], axis=1) #create dataframe

In [ ]:
df

In [ ]:
df['Label'].value_counts()

Since not every class has the same number of images, we need to sample them

In [ ]:
# category_samples = []
# for category in df['Label'].unique():
#     category_slice = df.query("Label == @category")
#     category_samples.append(category_slice.sample(100, random_state=1))
# df = pd.concat(category_samples, axis=0).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
# df['Label'].value_counts()

In [ ]:
# Falta fazer o exploration do dataset original, eliminar as classes que não queremos e juntar o scraping

Create the dataset with the images: Start by loading the images and spliting the dataset in training and validation using the ``tf.keras.utils.image_dataset_from_directory`` function and the validation_split=0.2 argument like shown in class

In [ ]:
# Define list of parameters:
image_size=(128, 128) #image size (height and width)
crop_to_aspect_ratio = True #respects the images aspect ratio when resizing. output image is cropped to fit the target aspect ratio if different
color_mode='rgb' #default color
batch_size=96 #no. of images loaded in each batch
shuffle=True #randomization of the instances so each batch has the representation of the maximum no. of classes
seed=28 #seed used for the shuffle


In [ ]:
# Create the train and validation datasets (80% for training and 20% validation):
data_dir_train = '/content/drive/MyDrive/IMS_DeepLearning/dataset_dishes/training'
data_dir_val = '/content/drive/MyDrive/IMS_DeepLearning/dataset_dishes/validation'

ds_train = tf.keras.utils.image_dataset_from_directory(
  data_dir_train,
    image_size=image_size,
    crop_to_aspect_ratio=crop_to_aspect_ratio,
    color_mode=color_mode,
    batch_size=batch_size,
    shuffle=shuffle,
    seed=seed)

ds_val = tf.keras.utils.image_dataset_from_directory(
  data_dir_val,
  image_size=image_size,
    crop_to_aspect_ratio=crop_to_aspect_ratio,
    color_mode=color_mode,
    batch_size=batch_size,
    shuffle=shuffle,
    seed=seed)

class_names = ds_train.class_names

In [ ]:
# Check object properties
print("\nObject's type:\t", type(ds_train))
print("Is it a tf.data.Dataset?\t R:",isinstance(ds_train, tf.data.Dataset))
print("Classes:", ds_train.class_names)

Checking the training dataset first nine images to get a feeling of the images' quality

In [ ]:
#Code source: https://www.tensorflow.org/tutorials/load_data/images

plt.figure(figsize=(10, 10))
for images, labels in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

Checking the number of images per class of the training and validation datasets

In [ ]:
classes = []
for (image,label) in tuple(ds_train.unbatch()):
    classes.append(label.numpy())
classes = pd.Series(classes)
count = classes.value_counts()
print(count)

In [ ]:
classes = []
for (image,label) in tuple(ds_val.unbatch()):
    classes.append(label.numpy())
classes = pd.Series(classes)
count = classes.value_counts()
print(count)

Notice the training and validation datasets are imbalanced

In [ ]:
for batch_x_train, batch_y_train in ds_train:
    print(batch_x_train.shape, batch_y_train.shape)
    break

In [ ]:
for batch_x_val, batch_y_val in ds_val:
    print(batch_x_val.shape, batch_y_val.shape)
    break

batch_x_train is a tensor with a batch of 64 images of shape 128x128 and color channel RGB 3. batch_y_train is a tensor with 64 images and corresponding labels

<a class="anchor" id="Datapreprocessing">

## 2.2. Data Preprocessing

</a>

Encoding the classes:

In [ ]:
encoding = layers.CategoryEncoding(num_tokens=6, output_mode="one_hot")

In [ ]:
ds_train_encoded = ds_train.map(lambda x, y: (x, encoding(y)))
batch_x_train_encoded, batch_y_train_encoded = next(iter(ds_train_encoded))

In [ ]:
ds_val_encoded = ds_val.map(lambda x, y: (x, encoding(y)))
batch_x_val_encoded, batch_y_val_encoded = next(iter(ds_val_encoded))

Rescaling the train and validation datatsets:

In [ ]:
rescaling = layers.Rescaling(1./255)

In [ ]:
ds_train_scaled = ds_train_encoded.map(lambda x, y: (rescaling(x), y))
batch_x_train_scaled, batch_y_train_scaled = next(iter(ds_train_scaled))


print(np.min(batch_x_train_scaled), np.max(batch_x_train_scaled))

In [ ]:
ds_val_scaled = ds_val_encoded.map(lambda x, y: (rescaling(x), y))
batch_x_val_scaled, batch_y_val_scaled = next(iter(ds_val_scaled))


print(np.min(batch_x_val_scaled), np.max(batch_x_val_scaled))

<a class="anchor" id="Dataaugmentation">

### *2.2.1 Data Augmentation*

</a>

Data augmentation techniques allow to include diversity and increase quality of the training dataset without the need for further data through the random transformations. Since the dataset used is small, data augmentation is particularly important to train the model.<br>
Following the notebook solved in class, we defined a data augmentation pipeline with the layers: 
-  `tf.keras.layers.RandomFlip`

-  `tf.keras.layers.RandomTranslation`

-  `tf.keras.layers.RandomRotation`

-  `tf.keras.layers.RandomZoom`

In [ ]:
augmentation = Sequential([layers.RandomFlip(seed=seed), 
                           layers.RandomRotation(0.1, seed=seed), 
                           layers.RandomZoom(0.1, seed=seed),
                           layers.RandomTranslation(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1))], 
                           name="augmentation")

Showing a sample image with the data augmentation pipeline applied (as the layers defined only implement data augmentation during the training, to test it, the training must be set to True):

In [ ]:
plt.imshow(tf.cast(augmentation(batch_x_train[0], training=True), tf.int32)) 

Using the function defined in class to visualize the pipeline defined applied on a set of images:

In [ ]:
def show_sample_batch(ds, augmentation, grid_size=(4, 4), figsize=(14, 10)):
    n_images = grid_size[0]
    # Get a batch via iteration
    iter_ = iter(ds)
    batch_x, batch_y = iter_.next()
    batch_x, batch_y = batch_x[0:n_images], batch_y[0:n_images]
    # Plot the images
    plt.figure(figsize=(14, 10))
    count = 0
    for i, (img, y) in enumerate(zip(batch_x, batch_y)):
        img_ = tf.cast(img, tf.int32)
        for j in range(grid_size[1]):
            # Prepare the image
            if j>0:
                img_ = tf.cast(augmentation(img_, training=True), tf.int32)
            ax = plt.subplot(grid_size[0], grid_size[1],  count + 1)        
            plt.imshow(img_)
            plt.title("{} {} image of class \"{}\"".format(
                img.shape, "original" if j==0 else "augmented", y), size=6)
            plt.axis("off")
            count+=1

show_sample_batch(ds_train, augmentation=augmentation)

[BACK TO TOC](#toc)
    
<a id='modelling'></a>

# <font color = 'darkblue'> 3. Modelling </font>

<a class="anchor" id="CNNv1">

## 3.0. Pre-modelling

</a>

Configure dataset for performance (https://www.tensorflow.org/tutorials/load_data/images):
- `Dataset.cache` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model.
- `Dataset.prefetch`overlaps data preprocessing and model execution while training.

In [ ]:
# AUTOTUNE = tf.data.AUTOTUNE

# ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
# ds_val = ds_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Shared parameters:
epochs=100
batch_size=96
suffle= True
input_shape = batch_x_train_scaled.shape
n_classes=6
learning_rate = 0.001

Calculate the evaluation weights for each class (since the dataset is no balanced)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(batch_y_train.numpy()), y= batch_y_train.numpy())

class_weights

In [ ]:
weights= dict(enumerate(class_weights.flatten(), 0))

In [ ]:
weights

In [ ]:
val_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(batch_y_val.numpy()), y= batch_y_val.numpy())

val_weights

In [ ]:
!pip install tensorflow-addons
import tensorflow_addons as tfa

Define function for confusion matrix inspired by: https://neptune.ai/blog/tensorboard-tutorial

In [ ]:
import itertools

def plot_confusion_matrix(cm, class_names):
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    threshold = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

    return figure

In [ ]:
def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)
    buf.seek(0)

    digit = tf.image.decode_png(buf.getvalue(), channels=4)
    digit = tf.expand_dims(digit, 0)

    return digit

<a class="anchor" id="CNNv1">

## 3.1. CNN_V1

</a>

In [ ]:
#Clear previous logs

#collab
!rm -rf /logs/ 


#jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
#try:
#    shutil.rmtree('logs')
#except:
#    pass

In [ ]:
# Architecture v1
cnn1 = Sequential([# Feature extraction                   
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.GlobalMaxPooling2D(),
                   # Classification 
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])

# Builds the DAG (comment if input_shape was already provided to the first layer)
cnn1.build(input_shape)
# Check network
cnn1.summary() 

In [ ]:
model=cnn1

In [ ]:
cnn1.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy']) 

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn1.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v1)
#Adding early stopping to help choosing the best number of epochs
#code from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/

#Train
history1 = cnn1.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

Plot learning curves

In [ ]:
# Create DataFrame object
df_hist1 = pd.DataFrame.from_dict(history1.history)
df_hist1["Epoch"] = np.arange(1, len(df_hist1) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist1.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv2">

## 3.2. CNN_V2 , Batch Normalization

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

### stabilizing input features of a deep network.
From [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift, by Sergey Ioffe, Christian Szegedy](https://arxiv.org/abs/1502.03167):

    "Training Deep Neural Networks is complicated by the fact that the distribution of each layer's inputs changes during training, as the parameters of the previous layers change. This slows down the training by requiring lower learning rates and careful parameter initialization, and makes it notoriously hard to train models with saturating nonlinearities. We refer to this phenomenon as internal covariate shift, and address the problem by normalizing layer inputs."


Adds ``BatchNormalization()`` layers before applying the non-linearities.

In [ ]:
# Architecture v2
cnn2_1 = Sequential([# The batch normalization layer
                   #layers.Input(input_shape[1:]), 
                   layers.BatchNormalization(),                   
                   layers.Conv2D(filters=32, kernel_size=(3, 3), #input_shape=input_shape[1:],
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),   
                   layers.Activation("relu"),
                   layers.GlobalMaxPooling2D(),
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])

# Builds the DAG (comment if input_shape was already provided to the first layer)
cnn2_1.build(input_shape)
# Check network
cnn2_1.summary() 

In [ ]:
model=cnn2_1

In [ ]:
cnn2_1.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy']) 

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn2_1.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v2)


#Train
history2_1 = cnn2_1.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

Plot learning curves

In [ ]:
# Create DataFrame object
df_hist2_1 = pd.DataFrame.from_dict(history2_1.history)
df_hist2_1["Epoch"] = np.arange(1, len(df_hist2_1) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist2_1.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

### BatchNormalization() layer to the head of the Sequential() model.

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
# Architecture v2.2
cnn2_2 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),     
                   #layers.Input(input_shape[1:]),
                   layers.Conv2D(filters=32, kernel_size=(3, 3), #input_shape=input_shape[1:],
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.GlobalMaxPooling2D(),
                   # Classification (use units=n_classes, activation="softmax" for multi-class problems)
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])

# Builds the DAG (comment if input_shape was already provided to the first layer)
cnn2_2.build(input_shape)
# Check network
cnn2_2.summary() 

In [ ]:
model=cnn2_2

In [ ]:
cnn2_2.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn2_2.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v2)

#Train

history2_2 = cnn2_2.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

Plot learning curves

In [ ]:
# Create DataFrame object
df_hist2_2 = pd.DataFrame.from_dict(history2_2.history)
df_hist2_2["Epoch"] = np.arange(1, len(df_hist2_2) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist2_2.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv3">

## 3.3. CNN_V3, data augmentation

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
for layer in augmentation.layers:
    print(layer.name)

In [ ]:
# Architecture v3
cnn3 = Sequential([# Add data augmentation
                   augmentation,                         
                   # Feature extraction
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.GlobalMaxPooling2D(),
                   # Classification (use units=n_classes, activation="softmax" for multi-class)
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG (comment if input_shape was already provided to the first layer)
cnn3.build(input_shape)
# Check network
cnn3.summary()  # alternatively use tf.keras.utils.plot_model(cnn1, show_shapes=True)

In [ ]:
learning_rate = 0.001
cnn3.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn3

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn3.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:

#Train
history3 = cnn3.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
df_hist3 = pd.DataFrame.from_dict(history3.history)
df_hist3["Epoch"] = np.arange(1, len(df_hist3) + 1, 1)
# 4.
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist3.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                  secondary_y = secondary_y,
                  kind="line", figsize=(6, 3), grid=True, legend=True,
                  ylabel="Cross-entropy", 
                  xlabel="Epoch", title="Learning curves",                  
                  color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.right_ax.set_ylim(0.45, 1.05)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv4">

## 3.4. CNN_V4 , without pre-processing

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
input_shape_nopre = batch_x_train_encoded.shape

In [ ]:
# Architecture v4
cnn4 = Sequential([       
                   #layers.Input(input_shape[1:]),
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.GlobalMaxPooling2D(),
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])

# Builds the DAG
cnn4.build(input_shape_nopre)
# Check network
cnn4.summary() 

In [ ]:
model=cnn4

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn4.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
# logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# # Defining the basic TensorBoard callback.
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
# def log_confusion_matrix(epoch, logs):
#     predictions_raw = model.predict(batch_x_val_scaled)
#     predictions = np.argmax(predictions_raw, axis=1)

#     cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
#     figure = plot_confusion_matrix(cm, class_names=class_names)
#     cm_image = plot_to_image(figure)

#     with file_writer_cm.as_default():
#         tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# # Defining the per-epoch callback.
# cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v1)
#Adding early stopping to help choosing the best number of epochs
#code from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/

cnn4.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

#Train
history4 = cnn4.fit(ds_train_encoded,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=ds_val_encoded,
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint])#, tensorboard_callback, cm_callback])

In [ ]:
# #Star tensorboard

# %tensorboard --logdir logs/image

Plot learning curves

In [ ]:
# Create DataFrame object
df_hist4 = pd.DataFrame.from_dict(history4.history)
df_hist4["Epoch"] = np.arange(1, len(df_hist4) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist4.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv5">

## 3.5. CNN_V5, layers

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
# Architecture v5
cnn5 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn5.build(input_shape)
# Check network
cnn5.summary() 

In [ ]:
learning_rate = 0.001
cnn5.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn5

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn5.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v5)

#Train

history5 = cnn5.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist5 = pd.DataFrame.from_dict(history5.history)
df_hist5["Epoch"] = np.arange(1, len(df_hist5) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax5 = df_hist5.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax5.right_ax.set_ylabel("Accuracy")
ax5.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax5.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv6">

## 3.6. CNN_V6, drop-out


</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

#### Drop-out 0.5

In [ ]:
#https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
#https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf
#drop-out rate: divide the number of nodes in the layer before dropout by the proposed dropout rate and use that as the number of nodes in the new 
#network that uses dropout. For example, a network with 100 nodes and a proposed dropout rate of 0.5 will require 200 nodes (100 / 0.5) when using dropout.

# Architecture v5
cnn6_1 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   #drop-out
                   layers.Dropout(0.5), 
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn6_1.build(input_shape)
# Check network
cnn6_1.summary() 

In [ ]:
learning_rate = 0.001
cnn6_1.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn6_1

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn6_1.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v6)
#Train

history6__1 = cnn6_1.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist6_1 = pd.DataFrame.from_dict(history6__1.history)
df_hist6_1["Epoch"] = np.arange(1, len(df_hist6_1) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax6_1 = df_hist6_1.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax6_1.right_ax.set_ylabel("Accuracy")
ax6_1.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax6_1.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

#### Drop-out 0.8

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
#https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
#https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf
#drop-out rate: divide the number of nodes in the layer before dropout by the proposed dropout rate and use that as the number of nodes in the new 
#network that uses dropout. For example, a network with 100 nodes and a proposed dropout rate of 0.5 will require 200 nodes (100 / 0.5) when using dropout.
# Architecture v5
cnn6_2 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   #drop-out
                   layers.Dropout(0.8), 
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn6_2.build(input_shape)
# Check network
cnn6_2.summary() 

In [ ]:
learning_rate = 0.001
cnn6_2.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])


In [ ]:
model=cnn6_2

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn6_2.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v6)

#Train

history6_2 = cnn6_2.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist6_2 = pd.DataFrame.from_dict(history6_2.history)
df_hist6_2["Epoch"] = np.arange(1, len(df_hist6_2) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax6_2 = df_hist6_2.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax6_2.right_ax.set_ylabel("Accuracy")
ax6_2.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax6_2.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

 #### Drop-out 0.2

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
#https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
#https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf
#drop-out rate: divide the number of nodes in the layer before dropout by the proposed dropout rate and use that as the number of nodes in the new 
#network that uses dropout. For example, a network with 100 nodes and a proposed dropout rate of 0.5 will require 200 nodes (100 / 0.5) when using dropout.

# Architecture v5
cnn6_3 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   #drop-out
                   layers.Dropout(0.5), 
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn6_3.build(input_shape)
# Check network
cnn6_3.summary() 

In [ ]:
learning_rate = 0.001
cnn6_3.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn6_3

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn6_3.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v6)

#Train

history6_3 = cnn6_3.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist6_3 = pd.DataFrame.from_dict(history6_3.history)
df_hist6_3["Epoch"] = np.arange(1, len(df_hist6_3) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax6_3 = df_hist6_3.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax6_3.right_ax.set_ylabel("Accuracy")
ax6_3.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax6_3.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv6">

## 3.6. CNN_V7, layer weight regularization

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
# Architecture v7

cnn7 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_regularizer=regularizers.L2(1e-4),
                                bias_regularizer=regularizers.L2(1e-4),
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn7.build(input_shape)
# Check network
cnn7.summary() 

In [ ]:
learning_rate = 0.001
cnn7.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn7

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn7.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v7)

#Train

history7 = cnn7.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist7 = pd.DataFrame.from_dict(history7.history)
df_hist7["Epoch"] = np.arange(1, len(df_hist7) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax7 = df_hist7.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax7.right_ax.set_ylabel("Accuracy")
ax7.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax7.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv4">

## 3.8. CNN_V8, Combining best methods

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

#### Drop-out with Batch Normalization:

In [ ]:
#https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/
#https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf
#drop-out rate: divide the number of nodes in the layer before dropout by the proposed dropout rate and use that as the number of nodes in the new 
#network that uses dropout. For example, a network with 100 nodes and a proposed dropout rate of 0.5 will require 200 nodes (100 / 0.5) when using dropout.

# Architecture v6
cnn8 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),
                   layers.Activation("relu"),                   
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.BatchNormalization(),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   #drop-out
                   layers.Dropout(0.5), 
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn8.build(input_shape)
# Check network
cnn8.summary() 

In [ ]:
learning_rate = 0.001
cnn8.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn8

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn8.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v5)
#Train

history8 = cnn8.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist8 = pd.DataFrame.from_dict(history8.history)
df_hist8["Epoch"] = np.arange(1, len(df_hist8) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax6 = df_hist8.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax6.right_ax.set_ylabel("Accuracy")
ax6.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax6.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

## CNN8_2 (Batch Normalization + Layers + Dropout)

In [ ]:
cnn8_2 = Sequential([# The batch normalization layer 
                   layers.BatchNormalization(),                           
                   # Feature extraction: first block
                   layers.Conv2D(filters=32, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: second block
                   layers.BatchNormalization(),     
                   layers.Conv2D(filters=64, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: third block
                   layers.BatchNormalization(),     
                   layers.Conv2D(filters=128, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),
                   layers.MaxPooling2D(pool_size=(2, 2)),
                   # Feature extraction: fourth (closing) block
                   layers.BatchNormalization(),     
                   layers.Conv2D(filters=256, kernel_size=(3, 3),
                                 kernel_regularizer=regularizers.L2(1e-4),
                                 bias_regularizer=regularizers.L2(1e-4),
                                 kernel_initializer=initializers.GlorotNormal(seed=seed)),
                   layers.Activation("relu"),                   
                   layers.GlobalMaxPooling2D(),
                   layers.Dropout(0.5),   
                   #drop-out
                   # Classification
                   layers.Dense(units=n_classes, activation="softmax",
                                kernel_regularizer=regularizers.L2(1e-4),
                                bias_regularizer=regularizers.L2(1e-4),
                                kernel_initializer=initializers.GlorotNormal(seed=seed))])
# Builds the DAG 
cnn8_2.build(input_shape)
# Check network
cnn8_2.summary()

In [ ]:
learning_rate = 0.001
cnn8_2.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])

In [ ]:
model=cnn8_2

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'cnn8_2.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training (v5)
#Train

history8_2 = cnn8_2.fit(ds_train_scaled,
                    batch_size=batch_size,
                    epochs=epochs,
                    shuffle=shuffle,
                    validation_data=(batch_x_val_scaled,batch_y_val_scaled),
                    class_weight=weights,
                    callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist8_2 = pd.DataFrame.from_dict(history8_2.history)
df_hist8_2["Epoch"] = np.arange(1, len(df_hist8_2) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax6 = df_hist8_2.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax6.right_ax.set_ylabel("Accuracy")
ax6.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax6.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

[BACK TO TOC](#toc)
    
<a id='transferlearning'></a>

# <font color = 'darkblue'> 4. Transferlearning </font>

<a class="anchor" id="CNNv4">

## 4.1. Transferlearning VGG16 model

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

Inspired by: https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
## Loading VGG16 model
input_shape_vgg=batch_x_train_scaled[0].shape
vgg16_base = VGG16(weights="imagenet", include_top=False, input_shape=input_shape_vgg)
vgg16_base.trainable = False

In [ ]:
ds_train_vgg16 = ds_train_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_train_vgg16, batch_y_train_vgg16 = next(iter(ds_train_vgg16))

In [ ]:
ds_val_vgg16 = ds_val_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_val_vgg16, batch_y_val_vgg16 = next(iter(ds_val_vgg16))

In [ ]:
vgg16_base.summary()

In [ ]:
vgg16 = Sequential([vgg16_base,
                    layers.Flatten(),
                    layers.Dense(50, activation='relu'),
                    layers.Dense(20, activation='relu'),
                    layers.Dense(units=6, activation="softmax")               
])

In [ ]:
model=vgg16

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'vgg16.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
learning_rate = 0.001
vgg16.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
  metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'],
  weighted_metrics= ['accuracy']
) 

In [ ]:
history_vgg16 = vgg16.fit(ds_train_vgg16 ,
                          batch_size=batch_size,
                          shuffle=shuffle,
                          epochs=epochs,
                          validation_data=ds_val_vgg16,
                          class_weight=weights,
                          callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_histvgg = pd.DataFrame.from_dict(history_vgg16.history)
df_histvgg["Epoch"] = np.arange(1, len(df_histvgg) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist1.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

<a class="anchor" id="CNNv4">

## 4.2. Transferlearning ResNet50 model

</a>

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

#### Preprocessing Resnet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

ds_train_resnet = ds_train_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_train_resnet, batch_y_train_resnet = next(iter(ds_train_resnet))

In [ ]:
ds_val_resnet = ds_val_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_val_resnet, batch_y_val_resnet = next(iter(ds_val_resnet))

#### First Resnet

In [ ]:
# Freeze the layers of the ResNet50 model

resnet = ResNet50(include_top = False, input_shape = (128,128,3),weights = 'imagenet', classifier_activation = 'softmax')

for layer in resnet.layers:
  layer.trainable = False

resnet = Sequential([resnet,
                   #layers.Flatten(),
                   layers.GlobalAveragePooling2D(),
                   layers.Dense(128, activation='relu'),
                   layers.Dense(units = 6, activation='softmax'),
])

print(resnet.summary())

resnet.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])
 


In [ ]:
model=resnet

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'resnet.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:
# Model training
#Adding early stopping to help choosing the best number of epochs
#code from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/

#Train
history_resnet = resnet.fit(ds_train_resnet , batch_size=batch_size, shuffle=shuffle, epochs=epochs, validation_data=ds_val_resnet, 
                          class_weight=weights, callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist_restnet = pd.DataFrame.from_dict(history_resnet.history)
df_hist_restnet["Epoch"] = np.arange(1, len(df_hist_restnet) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist_restnet.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

#### Testing layers -> Best model

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
resnet = ResNet50(include_top = False, input_shape = (128,128,3),weights = 'imagenet', classifier_activation = 'softmax')

# Freeze the layers of the ResNet50 model
for layer in resnet.layers:
  layer.trainable = False

resnet_2 = Sequential([resnet,
                   #layers.Flatten(),
                   layers.GlobalAveragePooling2D(),
                   layers.Dropout(0.2),
                   layers.BatchNormalization(),
                   layers.Dropout(0.2),
                   layers.Dense(128, activation='relu'),
                   layers.Dropout(0.1),
                   layers.Dense(units = 6, activation='softmax')
])

print(resnet_2.summary())

resnet_2.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])
 

In [ ]:
model=resnet_2

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'resnet_2.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

Might be underfitting since the lines are very flat
According to Andrew Ng, the best methods of dealing with an underfitting model is trying a bigger neural network (adding new layers or increasing the number of neurons in existing layers) or training the model a little bit longer.
https://www.mikulskibartosz.name/how-to-deal-with-underfitting-and-overfitting-in-deep-learning/#:~:text=According%20to%20Andrew%20Ng%2C%20the,model%20a%20little%20bit%20longer.

In [ ]:
#code inspired from: https://www.kaggle.com/code/imsparsh/food-classifier-mobilenetv2-resnet50-vgg16#Training-different-models


# Model training (v1)
#Adding early stopping to help choosing the best number of epochs
#code from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/

#Train
history_resnet_2 = resnet_2.fit(ds_train_resnet , batch_size=batch_size, shuffle=shuffle, epochs=epochs, validation_data=ds_val_resnet, 
                          class_weight=weights, callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist_restnet_2 = pd.DataFrame.from_dict(history_resnet_2.history)
df_hist_restnet_2["Epoch"] = np.arange(1, len(df_hist_restnet_2) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist_restnet_2.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

#### Bigger DropOut -> Does not improve

In [ ]:
# #Clear previous logs

# #collab
!rm -rf /logs/ 


# #jupyter using Windows based on https://www.machinelearningnuggets.com/tensorboard-tutorial/
# try:
#     shutil.rmtree('logs')
# except:
#     pass

In [ ]:
#code inspired from: https://www.kaggle.com/code/imsparsh/food-classifier-mobilenetv2-resnet50-vgg16#Training-different-models

resnet = ResNet50(include_top = False, input_shape = (128,128,3),weights = 'imagenet', classifier_activation = 'softmax')

# Freeze the layers of the ResNet50 model
for layer in resnet.layers:
  layer.trainable = False

resnet_3 = Sequential([resnet,
                   #layers.Flatten(),
                   layers.GlobalAveragePooling2D(),
                   layers.Dropout(0.8),
                   layers.BatchNormalization(),
                   layers.Dropout(0.5),
                   layers.Dense(128, activation='relu'),
                   layers.Dropout(0.1),
                   layers.Dense(units = 6, activation='softmax')
])

print(resnet_3.summary())

resnet_3.compile(
  optimizer= optimizers.Adam(learning_rate=learning_rate),
  loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics= [tfa.metrics.f_scores.F1Score(num_classes=6,average="weighted", name = 'f1_score'), 'accuracy'], weighted_metrics= ['accuracy'])
 


In [ ]:
model=resnet_3

Callbacks API setup (inspired in the class code):

In [ ]:
#checkpoint_filepath = '/content/drive/MyDrive/IMS_DeepLearning'
checkpoint_filepath = 'resnet_3.weights.best.hdf6'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_freq="epoch",
        save_best_only=True)

early_stopping = callbacks.EarlyStopping(
    monitor="val_f1_score",
    min_delta=0,
    mode="max",
    patience=3,
    restore_best_weights=True)


In [ ]:
logdir = "logs/image/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Defining the basic TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')

In [ ]:
def log_confusion_matrix(epoch, logs):
    predictions_raw = model.predict(batch_x_val_scaled)
    predictions = np.argmax(predictions_raw, axis=1)

    cm = sklearn.metrics.confusion_matrix(np.argmax(batch_y_val_scaled, axis=1), predictions)
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [ ]:
# Defining the per-epoch callback.
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

In [ ]:

# Model training (v1)
#Adding early stopping to help choosing the best number of epochs
#code from: https://www.geeksforgeeks.org/choose-optimal-number-of-epochs-to-train-a-neural-network-in-keras/

#Train
history_resnet_3 = resnet_3.fit(ds_train_resnet , batch_size=batch_size, shuffle=shuffle, epochs=epochs, validation_data=ds_val_resnet, 
                          class_weight=weights, callbacks =[early_stopping, model_checkpoint, tensorboard_callback, cm_callback])

In [ ]:
#Star tensorboard

%tensorboard --logdir logs/image

In [ ]:
# Create DataFrame object
df_hist_restnet_3 = pd.DataFrame.from_dict(history_resnet_3.history)
df_hist_restnet_3["Epoch"] = np.arange(1, len(df_hist_restnet_3) + 1, 1)
# Plot learning curves
secondary_y = ["f1_score", "val_f1_score"]
ax = df_hist_restnet_3.plot(x="Epoch", y=['loss', 'val_loss'] + secondary_y,
                   secondary_y = secondary_y,
                   kind="line", figsize=(6, 3), grid=True, legend=True,
                   ylabel="Cross-entropy", 
                   xlabel="Epoch", title="Learning curves",                  
                   color=['darkred', 'indianred', "darkblue", "royalblue"], alpha=0.75, fontsize=10)
ax.right_ax.set_ylabel("Accuracy")
ax.right_ax.legend(loc=(0.25, -0.45), framealpha=1.0)
ax.legend(loc=(0, -0.45), framealpha=1.0)
plt.show()

[BACK TO TOC](#toc)
    
<a id='transferlearning'></a>

# <font color = 'darkblue'> 4. Saving the Models </font>

In [ ]:
cnn1.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
cnn2.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
cnn3.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
#cnn4.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
cnn5.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
cnn6.save('/content/drive/MyDrive/IMS_DeepLearning/Models')
cnn7.save('/content/drive/MyDrive/IMS_DeepLearning/Models')

In [ ]:
vgg16.save('vgg16','/content/drive/MyDrive/IMS_DeepLearning/Models')
resnet.save('resnet','/content/drive/MyDrive/IMS_DeepLearning/Models')

[BACK TO TOC](#toc)
    
<a id='transferlearning'></a>

# <font color = 'darkblue'> 4. Testing with Test dataset </font>

<a class="anchor" id="Datapreprocessing">

## Importing 

</a>

In [ ]:
data_dir_test = '/content/drive/MyDrive/IMS_DeepLearning/dataset_dishes/test'
ds_test = tf.keras.utils.image_dataset_from_directory(
  data_dir_test,
  image_size=image_size,
    crop_to_aspect_ratio=crop_to_aspect_ratio,
    color_mode=color_mode,
    batch_size=batch_size,
    shuffle=shuffle,
    seed=seed)



In [ ]:
#img = ds_test[0]
# img = Image.fromarray(img, 'RGB')
# img.save('outfile.jpg')
# cv2.imwrite('myImage.png',img)

# img = image.load_img('outfile.jpg', target_size=(200, 200))
# img_tensor = image.img_to_array(img)
#img_tensor = np.expand_dims(img, axis=0)
#img_tensor /= 255.

<a class="anchor" id="Datapreprocessing">

## Data Preprocessing

</a>

Encoding the classes:

In [ ]:
ds_test_encoded = ds_test.map(lambda x, y: (x, encoding(y)))
batch_x_test_encoded, batch_y_test_encoded = next(iter(ds_test_encoded))

In [ ]:
rescaling = layers.Rescaling(1./255)

In [ ]:
ds_test_scaled = ds_test_encoded.map(lambda x, y: (rescaling(x), y))
batch_x_test_scaled, batch_y_test_scaled = next(iter(ds_test_scaled))



<a class="anchor" id="Datapreprocessing">

## Results with CNN1

</a>

In [ ]:
img = batch_x_test_encoded[0]

#predict = (vgg16.predict(img))

cnn1.evaluate(batch_x_test_scaled, batch_y_test_scaled)


In [ ]:
classes=batch_y_test_scaled.numpy()

In [ ]:
classes=np.argmax(classes, axis=1)

In [ ]:
cnn1.load_weights('cnn1.weights.best.hdf6')

In [ ]:
predicts= cnn1.predict(ds_test_scaled)

In [ ]:
predict_classes=np.argmax(predicts, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc= accuracy_score(classes,predict_classes)

In [ ]:
classes

In [ ]:
predict_classes

## Results with ResNet50

In [ ]:
ds_test_resnet = ds_test_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_test_resnet, batch_y_test_resnet = next(iter(ds_test_resnet))

In [ ]:
img = batch_x_test_resnet[0]

#predict = (vgg16.predict(img))

resnet_2.evaluate(batch_x_test_resnet, batch_y_test_resnet)


In [ ]:
classes=batch_y_test_resnet.numpy()

In [ ]:
classes=np.argmax(classes, axis=1)

In [ ]:
#resnet_2.load_weights('rest.weights.best.hdf6')

In [ ]:
predicts= resnet_2.predict(ds_test_resnet)

In [ ]:
predict_classes=np.argmax(predicts, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc= accuracy_score(classes,predict_classes)
acc

In [ ]:
print('Classes:',classes)
print('Predict:',predict_classes)

## CNN2

In [ ]:
img = batch_x_test_encoded[0]

#predict = (vgg16.predict(img))

cnn_2.evaluate(batch_x_test_scaled, batch_y_test_scaled)


In [ ]:
classes=batch_y_test_scaled.numpy()

In [ ]:
classes=np.argmax(classes, axis=1)

In [ ]:
cnn2.load_weights('cnn2.weights.best.hdf6')

In [ ]:
predicts= cnn2.predict(ds_test_scaled)

In [ ]:
predict_classes=np.argmax(predicts, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc= accuracy_score(classes,predict_classes)

In [ ]:
classes

In [ ]:
predict_classes

## Resultados Modelos

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn1.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn1.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn1.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn2_1.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn2_1.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn2_1.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn2_2.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn2_2.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn2_2.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn3.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn3.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn3.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn4.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn4.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn4.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn5.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn5.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn5.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn6_1.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn6_1.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn6_1.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn6_2.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn6_2.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn6_2.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn7.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn7.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn7.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = cnn8.evaluate(batch_x_train_scaled,batch_y_train_scaled, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn8.evaluate(batch_x_val_scaled,batch_y_val_scaled, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = cnn8.evaluate(batch_x_test_scaled,batch_y_test_scaled, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
ds_test_vgg16 = ds_test_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_test_vgg16, batch_y_test_vgg16 = next(iter(ds_test_vgg16))

In [ ]:
score, f1_score, acc, weighted_accuracy = vgg16.evaluate(batch_x_train_vgg16,batch_y_train_vgg16, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = vgg16.evaluate(batch_x_val_vgg16,batch_y_val_vgg16, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = vgg16.evaluate(batch_x_test_vgg16,batch_y_test_vgg16, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
ds_test_resnet = ds_test_encoded.map(lambda x, y: (preprocess_input(x), y))
batch_x_test_resnet, batch_y_test_resnet = next(iter(ds_test_resnet))

In [ ]:
score, f1_score, acc, weighted_accuracy = resnet.evaluate(batch_x_train_resnet,batch_y_train_resnet, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = resnet.evaluate(batch_x_val_resnet,batch_y_val_resnet, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = resnet.evaluate(batch_x_test_resnet,batch_y_test_resnet, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
score, f1_score, acc, weighted_accuracy = resnet_2.evaluate(batch_x_train_resnet,batch_y_train_resnet, verbose=0)

print("Training loss score: %0.3f" % (score))
print("Training f1-score: %0.3f" % (f1_score))
print("Training accuracy: %0.3f" % (acc))
print("Training weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = resnet_2.evaluate(batch_x_val_resnet,batch_y_val_resnet, verbose=0)

print("Validation loss score: %0.3f" % (score))
print("Validation f1-score: %0.3f" % (f1_score))
print("Validation accuracy: %0.3f" % (acc))
print("Validation weighted_accuracy: %0.3f" % (weighted_accuracy))

score, f1_score, acc, weighted_accuracy = resnet_2.evaluate(batch_x_test_resnet,batch_y_test_resnet, verbose=0)

print("Test loss score: %0.3f" % (score))
print("Test f1-score: %0.3f" % (f1_score))
print("Test accuracy: %0.3f" % (acc))
print("Test weighted_accuracy: %0.3f" % (weighted_accuracy))

print()

In [ ]:
resnet_2.evaluate(batch_x_test_resnet, batch_y_test_resnet)